In [1]:
from astropy.io import fits
import numpy as np
import gaia_tools.xmatch
from gaia_tools import query
from astropy.table import Table
import astropy.coordinates as acoords
from astropy import units as u
import matplotlib.pyplot as plt
from astroNN.gaia import extinction_correction
from astroNN.apogee import allstar, visit_spectra
from astroNN.apogee.chips import apogee_continuum
import h5py
import pandas as pd

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


Importamos los datos necesarios:
* apogeedr16: archivo fit con todas las estrellas (277371) de APOGEE DR16
* gaia_xmatched_allcolumns: tabla de pandas con las estrellas de Gaia (275020)

In [2]:
apogeedr16 = fits.getdata('/home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/aspcap/r12/l33/allStar-r12-l33.fits')
gaia_xmatched_allcolumns = fits.getdata('/home/anell/Desktop/Bovy/AnellExercises/match_gaia_apogee/apogeedr16_gaiadr_xmatch_allcolumns.fits')

In [3]:
ra_gaia_full = gaia_xmatched_allcolumns["ra"]
dec_gaia_full = gaia_xmatched_allcolumns["dec"]
parallax_gaia_full = gaia_xmatched_allcolumns["parallax"]
parallax_error_gaia_full = gaia_xmatched_allcolumns["parallax_error"]
phot_g_mean_mag = gaia_xmatched_allcolumns['phot_g_mean_mag']
bp_rp = gaia_xmatched_allcolumns['bp_rp']

ra_apogee_full = apogeedr16["ra"]
dec_apogee_full = apogeedr16["dec"]
id_apogee_full = apogeedr16["APOGEE_ID"]
location_id_full = apogeedr16["LOCATION_ID"]
field_apogee_full = apogeedr16['FIELD']
snr_full = apogeedr16["SNR"]
aspcap_Teff = apogeedr16["TEFF"]
aspcap_logg = apogeedr16['LOGG']

#nn_teff = astroNN_data_file['astroNN'][:, 0]
#nn_logg = astroNN_data_file['astroNN'][:, 1]
#nn_logg_error = astroNN_data_file['astroNN_error'][:, 1]

extinction_apogee = apogeedr16['AK_TARG']
extinction_apogee[extinction_apogee < 0.] = -9999  # assume corrupted extinction if negative extinction
corrected_K_apogee = extinction_correction(apogeedr16["K"], extinction_apogee) 

#Sanity:
vscatter = apogeedr16["vscatter"]  #APOGEE
visibility_periods_used = gaia_xmatched_allcolumns['visibility_periods_used'] #Gaia
starflag = apogeedr16['STARFLAG'] #APOGEE
metallicity = apogeedr16['FE_H']#APOGEE
astrometric_chi2 = gaia_xmatched_allcolumns['astrometric_chi2_al']  #Gaia
astrometric_n_good_obs_al = gaia_xmatched_allcolumns['astrometric_n_good_obs_al']    #Gaia

In [4]:
len(ra_gaia_full) , len(ra_apogee_full)

(467760, 473307)

In [5]:
idx_1, idx_2, sep = gaia_tools.xmatch.xmatch(apogeedr16, gaia_xmatched_allcolumns, colRA2='ra',colDec2='dec', epoch2=2015.5) 

/home/anell/anaconda3/envs/python37-astronn/lib/python3.7/site-packages/gaia_tools-0.2-py3.7.egg/gaia_tools/xmatch/__init__.py:424: UserWarning: You appear to be using a Gaia catalog, but are not setting the epoch to 2015. (DR1) or 2015.5 (DR2), which may lead to incorrect matches
  warnings.warn("You appear to be using a Gaia catalog, but are not setting the epoch to 2015. (DR1) or 2015.5 (DR2), which may lead to incorrect matches")


In [6]:
len(idx_1)

460724

Buscamos los índices respectivos de cada archivo (apogeedr16 y gaia_xmatched_allcolumns) para hacer el match entre los dos catálogos.

Establecemos el match

In [7]:
RA_GAIA_MATCH = ra_gaia_full[idx_2]
DEC_GAIA_MATCH = dec_gaia_full[idx_2]
PARALLAX_MATCH = parallax_gaia_full[idx_2]
PARALLAX_ERROR_MATCH = parallax_error_gaia_full[idx_2]
GMEAN_MATCH = phot_g_mean_mag[idx_2]
BPRP_MATCH = bp_rp[idx_2]

RA_APOGEE_MATCH = ra_apogee_full[idx_1]
DEC_APOGEE_MATCH = dec_apogee_full[idx_1]
ID_MATCH = id_apogee_full[idx_1]
LOCATION_ID_MATCH = location_id_full[idx_1]
SNR_MATCH = snr_full[idx_1]
ASPCAP_TEFF_MATCH = aspcap_Teff[idx_1]
ASPCAP_LOGG_MATCH = aspcap_logg[idx_1]
CORRECTED_K_APOGEE = corrected_K_apogee[idx_1]
EXTINCTION_APOGEE = extinction_apogee[idx_1]
FIELD_APOGEE = field_apogee_full[idx_1]

#NN_TEFF_MATCH = nn_teff[idx_1]
#NN_LOGG = nn_logg[idx_1]
#NN_LOGG_ERROR = nn_logg_error[idx_1]


#Sanity:
VSCATTER = vscatter[idx_1]
VISIBILITY_PERIODS_USED = visibility_periods_used[idx_2]
STARFLAG = starflag[idx_1]
METALLICITY = metallicity[idx_1]
ASTROMETRIC_CHI2 = astrometric_chi2[idx_2]
ASTROMETRIC_N_GOOD_OBS_AL = astrometric_n_good_obs_al[idx_2]


In [8]:
max(sep.arcsec), sep[:20].arcsec

(1.9990649449777727,
 array([0.13304631, 0.1259208 , 0.03395821, 0.03395821, 0.12158136,
        0.18640024, 0.08661348, 0.11506016, 0.0402598 , 0.04686412,
        0.06248871, 0.13748948, 0.11778222, 0.08577609, 0.08553986,
        0.18356386, 0.03732038, 0.05468106, 0.23173415, 0.17129068]))

In [9]:
CORRECTED_K_APOGEE[:20], min(CORRECTED_K_APOGEE)

(array([ 7.0716486, 10.663842 , 10.34071  , 10.37743  , 10.012492 ,
        10.605796 , 11.375968 , 10.027623 ,  6.70732  , 10.9656   ,
        11.955    , 11.112132 ,  7.472354 ,  9.17928  , 11.117315 ,
        11.709557 , 11.709608 ,  8.724201 , 12.078714 , 12.048187 ],
       dtype=float32),
 -9999.0)

In [10]:
PARALLAX_MATCH[:20], max(PARALLAX_MATCH)

(array([0.27083134, 5.48608731, 1.21955522, 1.21955522, 1.32528679,
        0.01954049, 0.42940465, 4.27935509, 0.31600402, 0.17999125,
        2.00381775, 0.77284839, 0.26711049, 3.1172177 , 0.02881986,
        2.3759284 , 0.16052258, 0.86658889, 0.17731153, 0.19316621]),
 768.5003653333918)

Guardamos el primer archivo fit

In [15]:
col = []
col.append(fits.Column(name='APOGEE_ID', array=ID_MATCH, format="18A"))
col.append(fits.Column(name='LOCATION_ID', array=LOCATION_ID_MATCH, format="J"))
col.append(fits.Column(name='FIELD_ID', array=LOCATION_ID_MATCH, format="J"))
col.append(fits.Column(name='RA', array=RA_APOGEE_MATCH, format='D'))
col.append(fits.Column(name='DEC', array=DEC_APOGEE_MATCH, format='D'))
col.append(fits.Column(name='RA_J2015.5', array=RA_GAIA_MATCH, format='D'))
col.append(fits.Column(name='DEC_J2015.5', array=DEC_GAIA_MATCH, format='D'))
col.append(fits.Column(name='parallax', array=PARALLAX_MATCH, format='D'))
col.append(fits.Column(name='parallax_error', array=PARALLAX_ERROR_MATCH, format='D'))
col.append(fits.Column(name='magnitude_K_corr', array=CORRECTED_K_APOGEE, format='D'))
col.append(fits.Column(name='visibility_periods_used', array=VISIBILITY_PERIODS_USED, format='I'))
col.append(fits.Column(name='astrometric_chi2_al', array=ASTROMETRIC_CHI2, format='E'))
col.append(fits.Column(name='astrometric_n_good_obs_al', array=ASTROMETRIC_N_GOOD_OBS_AL, format='J'))
col.append(fits.Column(name='phot_g_mean_mag', array=GMEAN_MATCH, format='D'))
col.append(fits.Column(name='bp_rp', array=BPRP_MATCH, format='D'))
col.append(fits.Column(name='Aspcap_teff', array=ASPCAP_TEFF_MATCH, format='D'))
col.append(fits.Column(name='Aspcap_Logg', array=ASPCAP_LOGG_MATCH, format='D'))
col.append(fits.Column(name='Fe/H', array=METALLICITY, format='D'))
col.append(fits.Column(name='SNR',array=SNR_MATCH, format='D'))

t = fits.BinTableHDU.from_columns(col)
t.writeto('/home/anell/Desktop/Bovy/AnellExercises/Fits_files/apogeedr16_gaiadr2_xmatch.fits')

In [11]:
#from astroNN.gaia import mag_to_fakemag, extinction_correction, fakemag_to_logsol

#fakemag, fakemag_error = mag_to_fakemag(CORRECTED_K_APOGEE, PARALLAX_MATCH, PARALLAX_ERROR_MATCH)
#logsol = fakemag_to_logsol(fakemag)

Aplicamos filtros 

In [8]:
good_idx , bad_idx=  [] , []
for i in range(len(PARALLAX_MATCH)):
    if ((PARALLAX_MATCH[i] != -9999.) and (PARALLAX_ERROR_MATCH[i] < 0.1) and (CORRECTED_K_APOGEE[i] != -9999.)
        #and (EXTINCTION_APOGEE[i] != -9999.)
        and (VISIBILITY_PERIODS_USED[i] > 10.) and (VSCATTER[i] < 1.) and (STARFLAG[i] == 0)):
        #and ((logsol[i] > 0) or (PARALLAX_MATCH[i] < 0))):
        good_idx.append(i)
    else:
        bad_idx.append(i)

In [9]:
#VOLVIENDO AL ORIGEN
len(good_idx), len(bad_idx)  #(85386, 186055)

(174093, 286631)

In [26]:
h5f = h5py.File('/home/anell/Desktop/Bovy/AnellExercises/Fits_files/apogeedr16_gaiadr2_filtered.h5', 'w')

asciiList1 = [n.encode("ascii", "ignore") for n in ID_MATCH[good_idx]]
asciiList2 = [n.encode("ascii", "ignore") for n in FIELD_APOGEE[good_idx]]


h5f.create_dataset('APOGEE_ID',data=asciiList1)
h5f.create_dataset('LOCATION_ID', data=LOCATION_ID_MATCH[good_idx])
h5f.create_dataset('FIELD_APOGEE', data=asciiList2)
h5f.create_dataset('RA', data=RA_APOGEE_MATCH[good_idx])
h5f.create_dataset('DEC', data=DEC_APOGEE_MATCH[good_idx])
h5f.create_dataset('SNR', data=SNR_MATCH[good_idx])
h5f.create_dataset('ASPCAP_TEFF', data=ASPCAP_TEFF_MATCH[good_idx])
h5f.create_dataset('ASPCAP_LOGG', data=ASPCAP_LOGG_MATCH[good_idx])
h5f.create_dataset('corrected_magnitude_K', data=CORRECTED_K_APOGEE[good_idx])  
h5f.create_dataset('parallax', data=PARALLAX_MATCH[good_idx])
h5f.create_dataset('parallax_err', data=PARALLAX_ERROR_MATCH[good_idx])
h5f.create_dataset('bp_rp', data=BPRP_MATCH[good_idx])
h5f.create_dataset('phot_g_mean_mag', data=GMEAN_MATCH[good_idx])
#h5f.create_dataset('NN_teff', data=NN_TEFF_MATCH[good_idx])
#h5f.create_dataset('NN_Logg', data=NN_LOGG[good_idx])
h5f.create_dataset('Fe/H', data=METALLICITY[good_idx])
h5f.close()

Conseguimos los espectros

In [10]:
with h5py.File('/home/anell/Desktop/Bovy/AnellExercises/Fits_files/apogeedr16_gaiadr2_filtered.h5') as F:  
    Parallax = np.array(F['parallax'])
    Parallax_error = np.array(F['parallax_err'])
    Bp_Rp = np.array(F['bp_rp'])
    Phot_g_mean_mag = np.array(F['phot_g_mean_mag'])
    Location_id = np.array(F['LOCATION_ID'])
    FIELD_ID = np.array(F['FIELD_APOGEE'])
    ID = np.array(F['APOGEE_ID'])
    Ra_apogee = np.array(F['RA'])
    Dec_apogee =  np.array(F['DEC'])
    Snr = np.array(F['SNR'])
    Aspcap_teff = np.array(F['ASPCAP_TEFF'])
    Aspcap_logg = np.array(F['ASPCAP_LOGG'])
    Corrected_magnitude_K = np.array(F['corrected_magnitude_K'])
    FE_H = np.array(F['Fe/H'])
    
Good_idx = []
for i in range(len(Parallax)):
    if ((Bp_Rp[i] != -9999.0) and (Phot_g_mean_mag[i] != -9999.) and (Aspcap_teff[i] > 0)
        and (Parallax_error[i] / Parallax[i] < 0.1)):
            Good_idx.append(i)

ID = ID[Good_idx]
location_id = Location_id[Good_idx]
FIELD_ID = FIELD_ID[Good_idx]
ra_apogee = Ra_apogee[Good_idx]
dec_apogee = Ra_apogee[Good_idx]
snr = Snr[Good_idx]
aspcap_teff = Aspcap_teff[Good_idx]
aspcap_logg = Aspcap_teff[Good_idx]
corrected_magnitude_K = Corrected_magnitude_K[Good_idx]
parallax = Parallax[Good_idx]
parallax_error = Parallax_error[Good_idx]
bp_rp = Bp_Rp[Good_idx]
phot_g_mean_mag = Phot_g_mean_mag[Good_idx]
Fe_H = FE_H[Good_idx]

#NN_teff = NN_Teff[Good_idx]
#NN_logg = NN_Logg[Good_idx]

    
apogee_id = []
field_id = []
for i in range(len(ID)):
    apogee_id.append(ID[i].decode("utf-8"))
    field_id.append(FIELD_ID[i].decode("utf-8"))    

/home/anell/anaconda3/envs/python37-astronn/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


In [11]:
len(parallax), len(Parallax)

(121100, 174093)

In [12]:
false_list = []
path_spectra = []

#apogee_id = apogee_id[:61776]
#location_id = location_id[:61776]

spec = np.zeros((len(apogee_id), 7514), dtype=np.float32)  #pregunta!

target_bit = [0, 1, 2, 3, 4, 5, 6, 7, 12]
def apstar_normalization(spectra, spectra_err, _spec_mask):
    return apogee_continuum(spectra=spectra, spectra_err=spectra_err, cont_mask=None, deg=2, dr=16, bitmask=_spec_mask, target_bit=target_bit)

for counter in range(0, len(apogee_id)):
    if location_id[counter] == 1:
        continue
    ap_path = visit_spectra(dr=16, apogee = apogee_id[counter],
                            location= location_id[counter], verbose=0)
    if ap_path is False:
        false_list.append(counter) #anell
        #pass
    else:
        path_spectra.append(ap_path) #anell
        apstar_file = fits.open(ap_path)
        nvisits = apstar_file[0].header['NVISITS']
        if nvisits == 1:
            _spec = apstar_file[1].data
            _spec_err = apstar_file[2].data
            _spec_mask = apstar_file[3].data
        else:
            _spec = apstar_file[1].data[1]
            _spec_err = apstar_file[2].data[1]
            _spec_mask = apstar_file[3].data[1]
        
        if not np.all(_spec==0.): #normaliza los espectros que tenga alguno de sus elementos distintos de 0
            _spec, _spec_err = apstar_normalization(_spec, _spec_err, _spec_mask) 
            spec[counter] = _spec
    print(counter)
5525
            
# save spec to fits file
hdu = fits.PrimaryHDU(spec)
hdu.writeto('/home/anell/Desktop/Bovy/AnellExercises/Fits_files/contspec_dr16.fits') 

/home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/aspcap/r12/l33/allStar-r12-l33.fits was found!
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
145
146
148
149
150
152
153
154
155
156
157
158
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
177
178
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
199
200
201
202
203
204
205
206
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
245
246
247
248
249
250
251
252
253
254
255
256
257
258

1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353


6801
6802
File corruption detected, astroNN is attempting to download again
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/apo25m/192-47_MGA/apStar-r12-2M03335915-0802375.fits
6803
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/apo25m/192-47_MGA/apStar-r12-2M03335965-0648487.fits
6804
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/apo25m/ALPHAPER/apStar-r12-2M03340475+4836074.fits
6805
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/apo25m/135+12/apStar-r12-2M03340573+7141182.fits
6806
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/apo25m/150-08

...

Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/285-33-O_TESS/asStar-r12-2M05105346-7240330.fits
14171
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/279-34-O/asStar-r12-2M05105456-6910186.fits
14172
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/289-33-O_TESS/asStar-r12-2M05105459-7607578.fits
14173
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/apo25m/160+06/apStar-r12-2M05105470+4757019.fits
14174
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/N1851/asStar-r12-2M05105620-3958436.fits
14175
Downloaded DR16 individua

Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/N1851/asStar-r12-2M05111520-3943203.fits
14215
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/N1851/asStar-r12-2M05111553-4018102.fits
14216
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/LMC7/asStar-r12-2M05111555-6235342.fits
14217
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/LMC6/asStar-r12-2M05111597-6510197.fits
14218
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/LMC7/asStar-r12-2M05111637-6205570.fits
14219
Downloaded DR16 individual visit file successful

Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/N1851/asStar-r12-2M05114344-4017118.fits
14258
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/apo25m/180-08-RV/apStar-r12-2M05114504+2527228.fits
14259
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/apo25m/173-04/apStar-r12-2M05114556+3344103.fits
14260
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/285-33-O_TESS/asStar-r12-2M05114677-7214567.fits
14261
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/N1851/asStar-r12-2M05114761-4041533.fits
14262
Downloaded DR16 individual visit

Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/LMC7/asStar-r12-2M05121153-6211382.fits
14302
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/279-34-O/asStar-r12-2M05121247-6841577.fits
14303
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/apo25m/160+06/apStar-r12-2M05121264+4750118.fits
14304
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/lco25m/279-34-O/asStar-r12-2M05121290-6834324.fits
14305
Downloaded DR16 individual visit file successfully to /home/anell/Desktop/Bovy/SDSS_LOCAL_SAS_MIRROR/dr16/apogee/spectro/redux/r12/stars/apo25m/160+06/apStar-r12-2M05121313+4914028.fits
14306
Downloaded DR16 individual visit file 

KeyboardInterrupt: 

In [17]:
false_list

[]

In [18]:
Path_spectra = []

for name in path_spectra:
    Path_spectra.append(name[25:])

In [9]:
spec = fits.open('/home/anell/Desktop/Bovy/AnellExercises/Fits_files/contspec_dr14_probando_rendimiento_1.fits')[0].data

h5f = h5py.File('/home/anell/Desktop/Bovy/AnellExercises/Fits_files/apogeedr14_gaiadr2_with_spectrum_probando_rendimiento_2.h5', 'w')

asciiList1 = [n.encode("ascii", "ignore") for n in apogee_id]
asciiList2 = [n.encode("ascii", "ignore") for n in Path_spectra]

h5f.create_dataset('APOGEE_ID',data=asciiList1)
h5f.create_dataset('LOCATION_ID', data=location_id)
h5f.create_dataset('spectra', data=spec)
h5f.create_dataset('RA', data=ra_apogee)
h5f.create_dataset('DEC', data=dec_apogee)
h5f.create_dataset('SNR', data=snr)
h5f.create_dataset('ASPCAP_TEFF', data=aspcap_teff)
h5f.create_dataset('ASPCAP_LOGG', data=aspcap_logg)
h5f.create_dataset('corrected_magnitude_K', data=corrected_magnitude_K)  # extinction corrected
h5f.create_dataset('parallax', data=parallax)
h5f.create_dataset('parallax_err', data=parallax_error)
h5f.create_dataset('bp_rp', data=bp_rp)
h5f.create_dataset('phot_g_mean_mag', data=phot_g_mean_mag)
h5f.create_dataset('NN_teff', data=NN_teff)
h5f.create_dataset('NN_Logg', data=NN_logg)
h5f.create_dataset('Fe/H', data=Fe_H)
h5f.create_dataset('Path_spectra', data=asciiList2)
h5f.close()

In [20]:
len(Path_spectra)

57696

In [3]:
NN_logg

array([2.820186 , 3.3790693, 3.1800456, ..., 1.6871586, 3.8434095,
       3.8966458], dtype=float32)